In [3]:
using FactCheck, ForwardDiff, Plots

In [17]:
function newton(f, x; tol_f = 1e-6, tol_x = 1e-3,
        maxiter = 1000, maxtime = 60)
    tol_g = tol_f^2
    g(x) = ForwardDiff.derivative(f, x)
    escala = max( abs(g(x)), abs(f(x)) )
    if abs(g(x)) < tol_g * escala
        return x, -1, f(x), g(x), 0, 0.0
    end
    d = -f(x)/g(x)
    iter = 0
    ef = 0
    start_time = time()
    el_time = 0.0
    while abs(f(x)) > tol_f || abs(d) > tol_x
        x = x + d
        if abs(g(x)) < tol_g * escala
            ef = -1
            break
        end
        d = -f(x)/g(x)
        iter += 1
        el_time = time() - start_time
        if iter >= maxiter
            ef = 1
            break
        end
        if el_time >= maxtime
            ef = 2
            break
        end
    end
    return x, ef, f(x), g(x), iter, el_time
end

newton (generic function with 1 method)

In [18]:
context("Exemplos básicos") do
    facts("f(x) = x^2 - 1") do
        f(x) = x^2-1
        x, ef, fx, fdx, iter, el_time = newton(f, 1.0)
        @fact x --> roughly(1.0, 1e-6)
        @fact ef --> 0
        @fact fx --> roughly(0.0, 1e-6)
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)
    end
    
    facts("f(x) = x^2") do
        f(x) = x^2
        x, ef, fx, fdx, iter, el_time = newton(f, 1.0)
        @fact x --> roughly(0.0, 1e-3)
        @fact ef --> 0
        @fact fx --> roughly(0.0, 1e-6)
        @fact fdx --> roughly(0.0, 2e-3)
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)
    end
end

context("Não quebre") do
    facts("f'(x) = 0") do
        f(x) = x^2 - 1
        x0 = 0.0
        x, ef, fx, fdx, iter, el_time = newton(f, x0)
        @fact ef --> -1 # -1 -> derivada zero
    end
    facts("Escala") do
        for p = [1e8; 1e-8]
            f(x) = p * (x^2 - 4)
            x, ef, fx, fdx, iter, el_time = newton(f, 1)
            @fact x --> roughly(2.0, 1e-3)
            @fact fx --> roughly(0.0, 1e-6)
            @fact ef --> 0
        end
    end
    facts("Longe da solução") do
        f(x) = x^2/(1+x^2)
        x0 = 2.0
        x, ef, fx, fdx, iter, el_time = newton(f, x0)
        @fact abs(x) --> greater_than(100)
        @fact ef --> not(0)
    end
end

context("Sem loop infinito") do
    f(x) = x^4
    x0 = 1.0
    facts("Iterações") do
        x, ef, fx, fdx, iter, el_time = newton(f, x0, maxiter=1)
        @fact ef --> not(0)
        @fact iter --> 1
    end
    
    facts("Tempo") do
        x, ef, fx, fdx, iter, el_time = newton(f, x0, maxtime=0.0)
        @fact ef --> not(0)
        @fact el_time --> greater_than(0.0)
    end
end

  > Exemplos básicos
f(x) = x^2 - 1
5 facts verified.
f(x) = x^2
6 facts verified.
  > Não quebre
f'(x) = 0
1 fact verified.
Escala
6 facts verified.
Longe da solução
2 facts verified.
  > Sem loop infinito
Iterações
2 facts verified.
Tempo
2 facts verified.


delayed_handler (generic function with 4 methods)

In [8]:
function secante(f, x; tol_f = 1e-6, tol_x = 1e-3,
        maxiter = 1000, maxtime = 60)
    d = 1
    iter = 0
    ef = 0
    start_time = time()
    el_time = 0.0
    while abs(f(x)) > tol_f || abs(d) > tol_x
        x1 = x + d
        m = (f(x1) - f(x))/(x1 - x)
        d = -f(x1)/m
        x = x1
        iter += 1
        el_time = time() - start_time
        if iter >= maxiter
            ef = 1
            break
        end
        if el_time >= maxtime
            ef = 2
            break
        end
    end
    return x, ef, f(x), iter, el_time
end

secante (generic function with 1 method)

In [9]:
x, ef, fx, iter, el_time = secante(x->x^2 - 2, 1)

(1.4142135620573204,0,-8.931455575122982e-10,6,0.002933979034423828)

In [16]:
context("Exemplos básicos") do
    facts("f(x) = x^2 - 1") do
        f(x) = x^2-1
        x, ef, fx, iter, el_time = secante(f, 1.0)
        @fact x --> roughly(1.0, 1e-6)
        @fact ef --> 0
        @fact fx --> roughly(0.0, 1e-6)
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)
    end
    
    facts("f(x) = x^2") do
        f(x) = x^2
        x, ef, fx, iter, el_time = secante(f, 1.0)
        @fact x --> roughly(0.0, 1e-3)
        @fact ef --> 0
        @fact fx --> roughly(0.0, 1e-6)
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)
    end
end

context("Não quebre") do
    facts("f'(x) = 0") do
        f(x) = x^2 - 1
        x0 = 0.0
        x, ef, fx, iter, el_time = secante(f, x0)
        @fact x --> roughly(1.0, 1e-6)
        @fact ef --> 0
        @fact fx --> roughly(0.0, 1e-6)
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)    end
    facts("Escala") do
        for p = [1e8; 1e-8]
            f(x) = p * (x^2 - 4)
            x, ef, fx, iter, el_time = secante(f, 1)
            @fact x --> roughly(2.0, 1e-3)
            @fact fx --> roughly(0.0, 1e-6)
            @fact ef --> 0
        end
    end
end

context("Sem loop infinito") do
    f(x) = x^4
    x0 = 1.0
    facts("Iterações") do
        x, ef, fx, iter, el_time = secante(f, x0, maxiter=1)
        @fact ef --> not(0)
        @fact iter --> 1
    end
    
    facts("Tempo") do
        x, ef, fx, iter, el_time = secante(f, x0, maxtime=0.0)
        @fact ef --> not(0)
        @fact el_time --> greater_than(0.0)
    end
end

  > Exemplos básicos
f(x) = x^2 - 1
5 facts verified.
f(x) = x^2
5 facts verified.
  > Não quebre
f'(x) = 0
5 facts verified.
Escala
6 facts verified.
  > Sem loop infinito
Iterações
2 facts verified.
Tempo
2 facts verified.


delayed_handler (generic function with 4 methods)

In [30]:
gr()
m = 2
b, a = -5m+10, 8m-17
f(x) = x < 2 ? x^2-1 : 3 + 4*(x-2) + a*(x-2)^2 + b*(x-2)^3
g(x) = ForwardDiff.derivative(f, x)
plot(f, -0.2, 3.5, leg=false)
plot!(x->f(3) + g(3)*(x-3), -0.2, 3.5)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,375.22 41.096,375.852 46.7589,376.354 52.4219,376.726 58.0848,376.967 63.7477,377.078 69.4106,377.058 75.0736,376.909 80.7365,376.629 86.3994,376.219 
 92.0624,375.678 97.7253,375.007 103.388,374.206 109.051,373.274 114.714,372.213 120.377,371.021 126.04,369.698 131.703,368.245 137.366,366.662 143.029,364.949 
 148.692,363.105 154.355,361.132 160.017,359.027 165.68,356.793 171.343,354.428 177.006,351.933 182.669,349.307 188.332,346.552 193.995,343.666 199.658,340.649 
 205.321,337.503 210.984,334.226 216.647,330.818 222.31,327.281 227.973,323.613 233.636,319.815 239.298,315.886 244.961,311.828 250.624,307.639 256.287,303.319 
 261.95,298.87 267.613,294.29 273.276,289.579 278.939,284.739 284.602,279.768 290.265,274.667 295.928,269.435 301.591,264.073 307.254,258.581 312.917,252.959 
 318.579,247.206 324.242,241.323 329.905,235.31 335.568,229.166 341.231,222.892 346.894,216.488 352.557,209.953 358.22,203.289 363.883,196.493 369.546,189.57 
 375.209,182.68 380.872,175.92 386.535,169.291 392.198,162.791 397.86,156.422 403.523,150.184 409.186,144.075 414.849,138.097 420.512,132.249 426.175,126.532 
 431.838,120.945 437.501,115.488 443.164,110.161 448.827,104.965 454.49,99.8988 460.153,94.9632 465.816,90.1578 471.479,85.4827 477.141,80.9379 482.804,76.5235 
 488.467,72.2393 494.13,68.0855 499.793,64.0619 505.456,60.1687 511.119,56.4057 516.782,52.7731 522.445,49.2707 528.108,45.8987 533.771,42.657 539.434,39.5455 
 545.097,36.5644 550.76,33.7136 556.422,30.9931 562.085,28.4028 567.748,25.9429 573.411,23.6133 579.074,21.414 584.737,19.345 590.4,17.4063 596.063,15.5979 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,349.1 41.096,345.613 46.7589,342.127 52.4219,338.64 58.0848,335.154 63.7477,331.667 69.4106,328.181 75.0736,324.694 80.7365,321.208 86.3994,317.721 
 92.0624,314.235 97.7253,310.748 103.388,307.262 109.051,303.775 114.714,300.289 120.377,296.802 126.04,293.316 131.703,289.829 137.366,286.343 143.029,282.856 
 148.692,279.37 154.355,275.883 160.017,272.397 165.68,268.91 171.343,265.424 177.006,261.937 182.669,258.451 188.332,254.964 193.995,251.478 199.658,247.991 
 205.321,244.505 210.984,241.018 216.647,237.532 222.31,234.045 227.973,230.559 233.636,227.073 239.298,223.586 244.961,220.1 250.624,216.613 256.287,213.127 
 261.95,209.64 267.613,206.154 273.276,202.667 278.939,199.181 284.602,195.694 290.265,192.208 295.928,188.721 301.591,185.235 307.254,181.748 312.917,178.262 
 318.579,174.775 324.242,171.289 329.905,167.802 335.568,164.316 341.231,160.829 346.894,157.343 352.557,153.856 358.22,150.37 363.883,146.883 369.546,143.397 
 375.209,139.91 380.872,136.424 386.535,132.937 392.198,129.451 397.86,125.964 403.523,122.478 409.186,118.991 414.849,115.505 420.512,112.018 426.175,108.532 
 431.838,105.045 437.501,101.559 443.164,98.0723 448.827,94.5858 454.49,91.0993 460.153,87.6128 465.816,84.1263 471.479,80.6398 477.141,77.1533 482.804,73.6669 
 488.467,70.1804 494.13,66.6939 499.793,63.2074 505.456,59.7209 511.119,56.2344 516.782,52.7479 522.445,49.2614 528.108,45.7749 533.771,42.2884 539.434,38.8019 
 545.097,35.3154 550.76,31.8289 556.422,28.3425 562.085,24.856 567.748,21.3695 573.411,17.883 579.074,14.3965 584.737,10.91 590.4,7.4235 596.063,3.93701 
 
 "/>

In [33]:
gr()
#f(x) = x < 2 ? x^2-1 : 3 + 4*(x-2) - (x-2)^3
function f(x)
    if x < 2
        return x^2 - 1
    else
        return 3 + 4*(x-2) - (x-2)^2
    end
end
g(x) = ForwardDiff.derivative(f, x)
plot(f, -0.2, 3.5, leg=false)
plot!(x->f(3) + g(3)*(x-3), -0.2, 3.5)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,375.22 41.096,375.852 46.7589,376.354 52.4219,376.726 58.0848,376.967 63.7477,377.078 69.4106,377.058 75.0736,376.909 80.7365,376.629 86.3994,376.219 
 92.0624,375.678 97.7253,375.007 103.388,374.206 109.051,373.274 114.714,372.213 120.377,371.021 126.04,369.698 131.703,368.245 137.366,366.662 143.029,364.949 
 148.692,363.105 154.355,361.132 160.017,359.027 165.68,356.793 171.343,354.428 177.006,351.933 182.669,349.307 188.332,346.552 193.995,343.666 199.658,340.649 
 205.321,337.503 210.984,334.226 216.647,330.818 222.31,327.281 227.973,323.613 233.636,319.815 239.298,315.886 244.961,311.828 250.624,307.639 256.287,303.319 
 261.95,298.87 267.613,294.29 273.276,289.579 278.939,284.739 284.602,279.768 290.265,274.667 295.928,269.435 301.591,264.073 307.254,258.581 312.917,252.959 
 318.579,247.206 324.242,241.323 329.905,235.31 335.568,229.166 341.231,222.892 346.894,216.488 352.557,209.953 358.22,203.289 363.883,196.493 369.546,189.57 
 375.209,182.68 380.872,175.92 386.535,169.291 392.198,162.791 397.86,156.422 403.523,150.184 409.186,144.075 414.849,138.097 420.512,132.249 426.175,126.532 
 431.838,120.945 437.501,115.488 443.164,110.161 448.827,104.965 454.49,99.8988 460.153,94.9632 465.816,90.1578 471.479,85.4827 477.141,80.9379 482.804,76.5235 
 488.467,72.2393 494.13,68.0855 499.793,64.0619 505.456,60.1687 511.119,56.4057 516.782,52.7731 522.445,49.2707 528.108,45.8987 533.771,42.657 539.434,39.5455 
 545.097,36.5644 550.76,33.7136 556.422,30.9931 562.085,28.4028 567.748,25.9429 573.411,23.6133 579.074,21.414 584.737,19.345 590.4,17.4063 596.063,15.5979 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,349.1 41.096,345.613 46.7589,342.127 52.4219,338.64 58.0848,335.154 63.7477,331.667 69.4106,328.181 75.0736,324.694 80.7365,321.208 86.3994,317.721 
 92.0624,314.235 97.7253,310.748 103.388,307.262 109.051,303.775 114.714,300.289 120.377,296.802 126.04,293.316 131.703,289.829 137.366,286.343 143.029,282.856 
 148.692,279.37 154.355,275.883 160.017,272.397 165.68,268.91 171.343,265.424 177.006,261.937 182.669,258.451 188.332,254.964 193.995,251.478 199.658,247.991 
 205.321,244.505 210.984,241.018 216.647,237.532 222.31,234.045 227.973,230.559 233.636,227.073 239.298,223.586 244.961,220.1 250.624,216.613 256.287,213.127 
 261.95,209.64 267.613,206.154 273.276,202.667 278.939,199.181 284.602,195.694 290.265,192.208 295.928,188.721 301.591,185.235 307.254,181.748 312.917,178.262 
 318.579,174.775 324.242,171.289 329.905,167.802 335.568,164.316 341.231,160.829 346.894,157.343 352.557,153.856 358.22,150.37 363.883,146.883 369.546,143.397 
 375.209,139.91 380.872,136.424 386.535,132.937 392.198,129.451 397.86,125.964 403.523,122.478 409.186,118.991 414.849,115.505 420.512,112.018 426.175,108.532 
 431.838,105.045 437.501,101.559 443.164,98.0723 448.827,94.5858 454.49,91.0993 460.153,87.6128 465.816,84.1263 471.479,80.6398 477.141,77.1533 482.804,73.6669 
 488.467,70.1804 494.13,66.6939 499.793,63.2074 505.456,59.7209 511.119,56.2344 516.782,52.7479 522.445,49.2614 528.108,45.7749 533.771,42.2884 539.434,38.8019 
 545.097,35.3154 550.76,31.8289 556.422,28.3425 562.085,24.856 567.748,21.3695 573.411,17.883 579.074,14.3965 584.737,10.91 590.4,7.4235 596.063,3.93701 
 
 "/>